In [1]:
import numpy as np
import astropy
import matplotlib.pyplot as plt
import pandas as pd
import camb
import astroquery
import healpy
import astropy.units as u
import sympy
import reproject
import treecorr
import Corrfunc
import math
from astropy.io import fits
from astropy.wcs import WCS
# %matplotlib inline
import healpy as hp
import scipy
import pymaster
import stilts_wrapper
import sys
import os

In [2]:
def data_extractor(cat,hdu=1):
    data=cat[hdu].data
    columns=cat[hdu].columns
    return data,columns

In [3]:
def mask(right_ra,left_ra,min_dec,max_dec,cat,ra_column='ra',dec_column='dec'):
    RA_LIMITS=(right_ra,left_ra)
    DEC_LIMITS=(min_dec, max_dec)
    dec_mask = (DEC_LIMITS[0] < cat.dec) & (cat.dec < DEC_LIMITS[1])
    if RA_LIMITS[0] < RA_LIMITS[1]:
        ra_mask = (RA_LIMITS[0] <= cat.ra) & (cat.ra < RA_LIMITS[1])
    else:
        ra_mask = (cat.ra <= RA_LIMITS[1]) | (RA_LIMITS[0] < cat.ra)
    mask=ra_mask & dec_mask
    return mask

In [4]:
def flux_gen_cling(cat,number):
    vals,bins,patches=plt.hist(cat.peak_flux,
    np.logspace(-3,np.log10(max(cat.peak_flux)),50),
    log=True,
    histtype='step',
    density=True,
    cumulative=True)
    plt.close()

    # vals,bins=np.histogram(array=cat.peak_flux,
    # bins=np.logspace(-3,np.log10(max(cat.peak_flux)),50)),


    random_numbers=np.random.uniform(0,1,number)
    
    CDF_numpy=np.zeros((len(bins)-1,2))
    CDF_numpy[:,0]=bins[:-1]
    CDF_numpy[:,1]=vals
    
    index_values=np.searchsorted(CDF_numpy[:,1],
    random_numbers,
    side='right')
    
    S_values=CDF_numpy[index_values,0]
    
    return S_values

In [5]:
def basic_rand_generator(number,model_cat=None,output='Fits'):
    #generates basic randoms across the entire sky, with an optional flux distribution.
    rand_ra=np.random.uniform(0,360,number)
    
    rand_sindec = np.random.uniform(-1, 1,number)
    rand_dec = np.arcsin(rand_sindec)
    rand_dec=rand_dec*180/np.pi

    if model_cat==None:
        rand_dict={'ra':rand_ra,'dec':rand_dec}
        rand_dataframe=pd.DataFrame(rand_dict,dtype=float)
        rand_dataframe.reset_index(drop=True,inplace=True)

        col1 = fits.Column(name='ra', format='E', array=rand_ra)
        col2 = fits.Column(name='dec', format='E', array=rand_dec)
        cols = fits.ColDefs([col1, col2])
        
        rand_hdu = fits.BinTableHDU.from_columns(cols)
        del rand_dict,rand_ra,rand_dec,rand_sindec,rand_fluxes

    
    else: 
        rand_fluxes=flux_gen_cling(model_cat,10**8)
        rand_dict={'ra':rand_ra,'dec':rand_dec,'sim_flux':rand_fluxes}
        rand_dataframe=pd.DataFrame(rand_dict,dtype=float)
        rand_dataframe.reset_index(drop=True,inplace=True)

        col1 = fits.Column(name='ra', format='E', array=rand_ra)
        col2 = fits.Column(name='dec', format='E', array=rand_dec)
        col3 = fits.Column(name='sim_flux', format='E', array=rand_fluxes)
        cols = fits.ColDefs([col1, col2,col3])
        
        rand_hdu = fits.BinTableHDU.from_columns(cols)
        del rand_dict,rand_ra,rand_dec,rand_sindec,rand_fluxes
    
    if output=='Fits':
        output_data=rand_hdu
    
    elif output=='Pandas':
        output_data=rand_dataframe
    
    return output_data

In [6]:
def noise_sim_pandas(random_cat,rms,sigma):
    
    wcs_RMS=WCS(rms[0].header)
    rms_data=rms[0].data
    
    positions_rands=astropy.coordinates.SkyCoord(random_cat.ra,
    random_cat.dec,
    unit=u.deg,
    frame='fk5')
    
    rms_pixel_x,rms_pixel_y=wcs_RMS.world_to_pixel(positions_rands)
    rms_pixel_x=np.int64(rms_pixel_x)
    rms_pixel_y=np.int64(rms_pixel_y)

    random_cat['x_pixel_int']=rms_pixel_x
    random_cat['y_pixel_int']=rms_pixel_y

    random_cat['local_rms']=rms_data[random_cat['y_pixel_int'],
    random_cat['x_pixel_int']].astype(float)
    
    random_cat['sim_flux_rms']=(random_cat['sim_flux']
    +(np.random.normal(0,random_cat['local_rms'],len(random_cat))))

    random_cat['flux_sigma_ratio']=random_cat['sim_flux_rms']/random_cat['local_rms']
    mask=random_cat['flux_sigma_ratio']>=sigma
    random_cat=random_cat[mask]
    random_cat.reset_index(drop=True,inplace=True)

    random_cat.drop(columns=['x_pixel_int'],inplace=True)
    random_cat.drop(columns=['y_pixel_int'],inplace=True)
    random_cat.drop(columns=['flux_sigma_ratio'],inplace=True)
    random_cat.drop(columns=['sim_flux'],inplace=True)

    return random_cat   

In [7]:
def noise_sim_fits(random_cat,rms,sigma):
    
    wcs_RMS=WCS(rms[0].header)
    rms_data=rms[0].data
    
    positions_rands=astropy.coordinates.SkyCoord(random_cat.ra,
    random_cat.dec,
    unit=u.deg,
    frame='fk5')
    
    random_cat=pd.DataFrame(random_cat)

    rms_pixel_x,rms_pixel_y=wcs_RMS.world_to_pixel(positions_rands)
    rms_pixel_x=np.int64(rms_pixel_x)
    rms_pixel_y=np.int64(rms_pixel_y)

    random_cat['x_pixel_int']=rms_pixel_x
    random_cat['y_pixel_int']=rms_pixel_y

    random_cat['local_rms']=rms_data[random_cat['y_pixel_int'],
    random_cat['x_pixel_int']].astype(float)
    
    random_cat['sim_flux_rms']=(random_cat['sim_flux']
    +(np.random.normal(0,random_cat['local_rms'],len(random_cat))))

    random_cat['flux_sigma_ratio']=random_cat['sim_flux_rms']/random_cat['local_rms']
    mask=random_cat['flux_sigma_ratio']>=sigma
    random_cat=random_cat[mask]
    random_cat.reset_index(drop=True,inplace=True)

    random_cat.drop(columns=['x_pixel_int'],inplace=True)
    random_cat.drop(columns=['y_pixel_int'],inplace=True)
    random_cat.drop(columns=['flux_sigma_ratio'],inplace=True)
    random_cat.drop(columns=['sim_flux'],inplace=True)

    

    return random_cat   

In [8]:
def healpixer_fits(cat,ring=True,level=7,remove=False):
    intermediate_path='./Intermediate_healpixer_fits.fits'
    fits.writeto(intermediate_path,cat,overwrite=True)
    if ring==True:
        os.system('stilts tskymap in='
        +intermediate_path
        +' lon=ra lat=dec tiling=healpixring'
        +str(level)
        +' ocmd="healpixmeta -csys C"'
        +' ofmt=fits-healpix'
        +' out='+intermediate_path
        +' count=true')
    
    else:
        os.system('stilts tskymap in='
        +intermediate_path
        +' lon=ra lat=dec tiling=hpx'
        +str(level)
        +' ocmd="healpixmeta -csys C"'
        +' ofmt=fits-healpix'
        +' out='+intermediate_path
        +' count=true')
    
    output=fits.open(intermediate_path)
    
    if remove==True:
        os.system('rm '+intermediate_path)
    
    output_data=output[1].data
    
    return output_data

In [9]:
def healpixer_Pandas(cat,ring=True,level=7,remove=False):
    intermediate_path='./Intermediate_healpixer_pandas'
    cat.to_csv(intermediate_path+'.csv',index=False)
    if ring==True:
        os.system('stilts tskymap in='
        +intermediate_path+'.csv'
        +' lon=ra lat=dec tiling=healpixring'
        +str(level)
        +' ocmd="healpixmeta -csys C"'
        +' ofmt=fits-healpix'
        +' out='+intermediate_path+'.fits'
        +' count=true')
    
    else:
        os.system('stilts tskymap in='
        +intermediate_path+'.csv'
        +' lon=ra lat=dec tiling=hpx'
        +str(level)
        +' ocmd="healpixmeta -csys C"'
        +' ofmt=fits-healpix'
        +' out='+intermediate_path+'.fits'
        +' count=true')

    output=fits.open(intermediate_path+'.fits')
    
    if remove==True:
        os.system('rm '+intermediate_path+'.fits')
    
    output_data=output[1].data
    return output_data    

In [10]:
def frac_weights_healpix(rands_noise,rands_no_noise):
    weights=(rands_noise.count/rands_no_noise.count)
    pixels=rands_noise.PIXEL
    dict={'PIXEL':pixels,'weights':weights}
    dataframe=pd.DataFrame(dict)
    col1 = fits.Column(name='PIXEL', format='E', array=dataframe.PIXEL)
    col2 = fits.Column(name='weights', format='E', array=dataframe['weights'])
    cols = fits.ColDefs([col1, col2])
    new_cat_healpix=fits.BinTableHDU.from_columns(cols)
    return new_cat_healpix.data

In [11]:
def healpix_len_cruncher(cat_healpix,rand_healpix):
    cat_dataframe=pd.DataFrame(cat_healpix)
    dummy_dict={cat_dataframe.columns[0]:np.zeros(1),'count':np.zeros(1)}
    dummy_row=pd.DataFrame(dummy_dict,dtype=int)
    for i in range(len(rand_healpix)):
        if cat_dataframe.iloc[i,0]!=rand_healpix.PIXEL[i]:
            cat_dataframe=pd.concat([cat_dataframe,dummy_row])
            cat_dataframe.iloc[-1,0]=rand_healpix.PIXEL[i]
            cat_dataframe.iloc[-1,1]=0
            cat_dataframe.sort_values('PIXEL',axis=0,inplace=True,ignore_index=True)
    col1 = fits.Column(name='PIXEL', format='E', array=cat_dataframe.PIXEL)
    col2 = fits.Column(name='count', format='E', array=cat_dataframe['count'])
    cols = fits.ColDefs([col1, col2])
    new_cat_healpix=fits.BinTableHDU.from_columns(cols)
    return new_cat_healpix.data

In [12]:
def healpix_len_cruncher_V2(healpix_1,healpix_2):
    if len(healpix_1)>len(healpix_2):
        cat_to_expand=healpix_2
        model_cat=healpix_1
    if len(healpix_2)>len(healpix_1):
        cat_to_expand=healpix_1
        model_cat=healpix_2
    cat_dataframe=pd.DataFrame(cat_to_expand)
    dummy_dict={cat_dataframe.columns[0]:np.zeros(1),'count':np.zeros(1)}
    dummy_row=pd.DataFrame(dummy_dict,dtype=int)
    for i in range(len(model_cat)):
        if cat_dataframe.iloc[i,0]!=model_cat.PIXEL[i]:
            cat_dataframe=pd.concat([cat_dataframe,dummy_row])
            cat_dataframe.iloc[-1,0]=model_cat.PIXEL[i]
            cat_dataframe.iloc[-1,1]=0
            cat_dataframe.sort_values('PIXEL',axis=0,inplace=True,ignore_index=True)
    col1 = fits.Column(name='PIXEL', format='E', array=cat_dataframe.PIXEL)
    col2 = fits.Column(name='count', format='E', array=cat_dataframe['count'])
    cols = fits.ColDefs([col1, col2])
    new_cat_healpix=fits.BinTableHDU.from_columns(cols)
    return new_cat_healpix.data

In [13]:
def healpix_len_cruncher_V3(cat_to_expand,model_cat):
    if len(cat_to_expand)>len(model_cat):
        print('error: cat to expand larger than model cat')
    else:
        cat_dataframe=pd.DataFrame(cat_to_expand)
        dummy_dict={cat_dataframe.columns[0]:np.zeros(1),'count':np.zeros(1)}
        dummy_row=pd.DataFrame(dummy_dict,dtype=int)
        for i in range(len(model_cat)):
            if cat_dataframe.iloc[i,0]!=model_cat.PIXEL[i]:
                cat_dataframe=pd.concat([cat_dataframe,dummy_row])
                cat_dataframe.iloc[-1,0]=model_cat.PIXEL[i]
                cat_dataframe.iloc[-1,1]=0
                cat_dataframe.sort_values('PIXEL',axis=0,inplace=True,ignore_index=True)
        col1 = fits.Column(name='PIXEL', format='E', array=cat_dataframe.PIXEL)
        col2 = fits.Column(name='count', format='E', array=cat_dataframe['count'])
        cols = fits.ColDefs([col1, col2])
        new_cat_healpix=fits.BinTableHDU.from_columns(cols)
        return new_cat_healpix.data

In [14]:
def dataframe_healpix_healpixer(dataframe):
    col1 = fits.Column(name='PIXEL', format='E', array=dataframe.PIXEL)
    col2 = fits.Column(name='count', format='E', array=dataframe['count'])
    cols = fits.ColDefs([col1, col2])
    new_cat_healpix=fits.BinTableHDU.from_columns(cols)
    return new_cat_healpix.data

In [15]:
def healpix_len_cruncher_V4_remove_rows(cat_to_shrink,model_cat):
    if len(cat_to_shrink)<len(model_cat):
        print('error: cat to expand smaller than model cat')
    else:
        cat_dataframe=pd.DataFrame(cat_to_shrink)
        for i in range(len(cat_to_shrink)):
            if cat_dataframe.iloc[i,0]!=model_cat.PIXEL[i]:
                cat_dataframe.iloc[i,0]=np.nan
        cat_dataframe=cat_dataframe.dropna(inplace=True,subset=['PIXEL'])
        cat_dataframe.sort_values('PIXEL',axis=0,inplace=True,ignore_index=True)
        col1 = fits.Column(name='PIXEL', format='E', array=cat_dataframe.PIXEL)
        col2 = fits.Column(name='count', format='E', array=cat_dataframe['count'])
        cols = fits.ColDefs([col1, col2])
        new_cat_healpix=fits.BinTableHDU.from_columns(cols)
        return new_cat_healpix.data

In [16]:
def overdensity(cat,weights):
    cat_dataframe=pd.DataFrame(cat)
    weight_dataframe=pd.DataFrame(weights)
    cat_dataframe['weights']=weight_dataframe['weights']
    mask=cat_dataframe['weights']>=0.5
    cat_dataframe=cat_dataframe[mask]
    cat_dataframe['weighted_count']=cat_dataframe['count']/cat_dataframe['weights']
    mean_weighted_value=np.nanmean(cat_dataframe['weighted_count'])
    cat_dataframe['overdensity']=(cat_dataframe['count']/(mean_weighted_value*cat_dataframe['weights']))-1
    col1 = fits.Column(name='PIXEL', format='E', array=cat_dataframe.PIXEL)
    col2 = fits.Column(name='overdensity', format='E', array=cat_dataframe['overdensity'])
    cols = fits.ColDefs([col1, col2])
    new_cat_healpix=fits.BinTableHDU.from_columns(cols)
    return new_cat_healpix.data,new_cat_healpix

In [17]:
def blanker(cat,level=7,fill_value=np.nan):
        cat_dataframe=pd.DataFrame(cat)
        dummy_dict={cat_dataframe.columns[0]:np.zeros(1),'count':fill_value*np.zeros(1)}
        dummy_row=pd.DataFrame(dummy_dict,dtype=int)
        npix=hp.nside2npix(hp.order2nside(level))
        dummy_cat_dict={cat_dataframe.columns[0]:np.arange(npix)+1,'count':fill_value*np.ones(npix)}
        model_cat=pd.DataFrame(dummy_cat_dict)
        for i in range(len(cat)):
            if cat_dataframe.iloc[i,0]!=model_cat.PIXEL[i]:
                cat_dataframe=pd.concat([cat_dataframe,dummy_row])
                cat_dataframe.iloc[-1,0]=model_cat.PIXEL[i]
                #cat_dataframe.iloc[-1,1]=0
                cat_dataframe.sort_values('PIXEL',axis=0,inplace=True,ignore_index=True)
        col1 = fits.Column(name='PIXEL', format='E', array=cat_dataframe.PIXEL)
        col2 = fits.Column(name='count', format='E', array=cat_dataframe['count'])
        cols = fits.ColDefs([col1, col2])
        new_cat_healpix=fits.BinTableHDU.from_columns(cols)
        return new_cat_healpix.data

In [18]:
#Basic importiing, generation.
catalogue=astropy.io.fits.open("../Data/XG_IDR2_4Parts_170-231MHz_comp_rescaled.fits")
rms=astropy.io.fits.open("../Data/XG_IDR2_4Parts_170-231MHz_rms.fits")
cat_data,cat_cols=data_extractor(catalogue)
rands=basic_rand_generator(10**8,model_cat=cat_data,output='Pandas')


#Masking to appropriate area.
cat_mask=mask(339,83,-30,-10,cat_data)
rands_mask=mask(339,83,-30,-10,rands)
rands=rands[rands_mask]
rands_no_noise=rands.copy(deep=True)
cat_data=cat_data[cat_mask]


#Simulating the noise properties of the catalogue.
rands=noise_sim_pandas(rands,rms,5)


#Healpix making.
rands_healpix=healpixer_Pandas(rands,ring=True,level=7,remove=False)
rands_no_noise_healpix=healpixer_Pandas(rands_no_noise,ring=True,level=7,remove=False)
cat_healpix=healpixer_fits(cat_data,ring=True,level=7,remove=False)
cat_healpix=healpix_len_cruncher_V3(cat_healpix,rands_no_noise_healpix)
rands_healpix=healpix_len_cruncher_V3(rands_healpix,rands_no_noise_healpix)

#computing the weights
weights=frac_weights_healpix(rands_healpix,rands_no_noise_healpix)

#computing the overdensity
overdensity,overdensity_fits=overdensity(cat_healpix,weights)

In [ ]:
len(overdensity)

9533

In [ ]:
overdensity_test=blanker(overdensity)
len(overdensity_test)

19066

In [ ]:
hp.nside2npix(hp.order2nside(7))

196608

In [ ]:
hp.fitsfunc.write_map('../Data/test_overdensity.fits',overdensity_test,overwrite=True,fits_IDL=False,column_names=['overdensity'],partial=True)

TypeError: bad number of pixels

In [ ]:
overdensity_test.shape

(19062,)

In [ ]:
overdensity_test.PIXEL

array([1.00000e+00, 2.00000e+00, 3.00000e+00, ..., 1.47709e+05,
       1.47710e+05, 1.47711e+05], dtype=float32)